# Data Connection
- LLMと外部データを接続するための機能

## RAG (Retreieval Augumented Generation)
- LLMはある時点までのデータしか知らない
- そこでプロンプトに前提知識(context)を組み込んであげる
- RAGでは入力に関係しそうな文章を検索してcontextに含める
    1. 文章をベクトル化しておく(Open AI Embedding API等を使って)
    1. 入力をベクトル化する
    1. 入力と近いベクトルを持つ文章を検索する(Vector Store や　Web検索)
    1. 検索した文章をcontextとしてプロンプトに入れる
    1. LLMに回答させる
- LangchainのData Connection
    - 文章をベクトル化してVector Storeに保存
    - 入力のテキストと近い文章を検索してプロンプトにcontextを入れる

## Data Conection
| 種類                  | 機能                                     |
| --------------------- | ---------------------------------------- |
| Document Loaders      | データソースからドキュメントを読み込む   |
| Document Transformers | ドキュメントに何らかの変換をかける       |
| Text Embedding Models | ドキュメントのベクトル化                 |
| Vector Stores         | ベクトル化したドキュメントの保存         |
| Retrievers            | 入力テキストと近いドキュメントを検索する |


## Document Loaders
- 様々なデータソースの読み込み
- [非常に多くのデータソースに対応](https://integrations.langchain.com/)
    - CSV、HTML、JSON、PDF、git repo, AWS S3, Wikipedia, YouTube...

In [1]:
# github上のgit repoからデータ取得する例
from langchain.document_loaders import GitLoader

def file_filter(file_path):
    return file_path.endswith('.mdx') or file_path.endswith('.md')

loader = GitLoader(
    clone_url='https://github.com/langchain-ai/langchain',
    repo_path='./langchain',
    branch='master',
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

431


In [2]:
for k, v in raw_docs[0].dict().items():
    sep = '-'*30
    print('\n' + sep + k + sep)
    print(v)


------------------------------page_content------------------------------
# Migrating

## 🚨Breaking Changes for select chains (SQLDatabase) on 7/28/23

In an effort to make `langchain` leaner and safer, we are moving select chains to `langchain_experimental`.
This migration has already started, but we are remaining backwards compatible until 7/28.
On that date, we will remove functionality from `langchain`.
Read more about the motivation and the progress [here](https://github.com/langchain-ai/langchain/discussions/8043).

### Migrating to `langchain_experimental`

We are moving any experimental components of LangChain, or components with vulnerability issues, into `langchain_experimental`.
This guide covers how to migrate.

### Installation

Previously:

`pip install -U langchain`

Now (only if you want to access things in experimental):

`pip install -U langchain langchain_experimental`

### Things in `langchain.experimental`

Previously:

`from langchain.experimental import ...`

Now

## Document Transformers
- Document Loaderで読み込んだデータ(ドキュメント)に何らかの変換をかける
- 様々なものが存在(下記も一部)
| Document Transformer          | 概要                                                                           |
| ----------------------------- | ------------------------------------------------------------------------------ |
| CharacterTextSplitter         | ドキュメントをある程度の長さでチャンク分割(指定サイズより大きくなる場合がある) |
| ecursiveCharacterTextSplitter | ドキュメントをチャンクサイズの制限を下回るまで再起的に分割                     |
| Html2TextTransformer          | HTMLをプレーンテキストに変換                                                   |
| EmbeddingsRedundantFilter     | 類似したドキュメントを除外                                                     |
| OpenAIMetadataTagger          | メタデータ抽出                                                                 |
| DoctranTextTranslator         | ドキュメント翻訳                                                               |
| DoctranQATransformer          | ユーザーの質問と関連しやすくなるよう、ドキュメントからQ&Aを生成する            |

In [5]:
# ドキュメントをある程度の長さのチャンクに分割
from langchain.text_splitter import CharacterTextSplitter

# CharacterTextSplitterは区切り文字(デフォルト\n\n)で分割するため
# 文章途中など変な場所で区切られることはなさそう。
# よって overlapは0でも問題ないと思われる。
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
)

docs = text_splitter.split_documents(raw_docs)
len(docs)

Created a chunk of size 1588, which is longer than the specified 1000
Created a chunk of size 1103, which is longer than the specified 1000
Created a chunk of size 6803, which is longer than the specified 1000
Created a chunk of size 3302, which is longer than the specified 1000
Created a chunk of size 1851, which is longer than the specified 1000
Created a chunk of size 1639, which is longer than the specified 1000
Created a chunk of size 9269, which is longer than the specified 1000
Created a chunk of size 2579, which is longer than the specified 1000
Created a chunk of size 1359, which is longer than the specified 1000
Created a chunk of size 1736, which is longer than the specified 1000
Created a chunk of size 1009, which is longer than the specified 1000
Created a chunk of size 1080, which is longer than the specified 1000
Created a chunk of size 42230, which is longer than the specified 1000
Created a chunk of size 1386, which is longer than the specified 1000
Created a chunk of 

1392

In [6]:
for k, v in docs[0].dict().items():
    sep = '-'*30
    print('\n' + sep + k + sep)
    print(v)


------------------------------page_content------------------------------
# Migrating

## 🚨Breaking Changes for select chains (SQLDatabase) on 7/28/23

In an effort to make `langchain` leaner and safer, we are moving select chains to `langchain_experimental`.
This migration has already started, but we are remaining backwards compatible until 7/28.
On that date, we will remove functionality from `langchain`.
Read more about the motivation and the progress [here](https://github.com/langchain-ai/langchain/discussions/8043).

### Migrating to `langchain_experimental`

We are moving any experimental components of LangChain, or components with vulnerability issues, into `langchain_experimental`.
This guide covers how to migrate.

### Installation

Previously:

`pip install -U langchain`

Now (only if you want to access things in experimental):

`pip install -U langchain langchain_experimental`

### Things in `langchain.experimental`

Previously:

`from langchain.experimental import ...`

Now

## Text Embedding Models
- テキストのベクトル化
- 元々 langchain本体に入っていたものが、OpenAI専用パッケージ `langchain_openai` に外出しされた
    ```python
    # Before
    from langchain.embeddings.openai import OpenAIEmbeddings

    # After
    from langchain_openai.embeddings import OpenAIEmbeddings
    # これでもOK
    from langchain_openai import OpenAIEmbeddings

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings

# デフォルトで使用されるEmbeddingモデルが text-embedding-ada-002
embeddings = OpenAIEmbeddings()

query = 'AWSのS3からデータを読み込むためのDocumentLoaderはありますか？'
vec = embeddings.embed_query(query)
print(len(vec))
print(vec)

1536
[-0.015850006973008344, -0.0007096511293475243, 0.016529099430703905, -0.006125409404933484, -0.014858532171037338, 0.037295731327079897, -0.013853475892441437, -0.021391396584926888, 0.0018369438407810548, 0.0039964563661368645, 0.012828047864569463, 0.016461189253611806, 0.018661448071487383, -0.010186379414853045, -0.006787524318356018, -0.017085953942162703, 0.017031626173018042, -0.002407380993017926, 0.0021272557034644614, -0.0427284672633542, 0.008332458961121508, -0.005948845552197147, -0.004271488485540676, -0.006553237560149432, -0.015551206105357792, 0.008488650133259232, 0.013649749086455311, -0.03830078853699834, 0.01978873950235242, -0.007857094240734616, 0.05174680988614498, -0.017153864119254802, 0.005062630686528603, -0.021079014240651436, -0.0013403577721294718, -0.026416677046583854, 0.0001150212006210799, -0.005592322244737614, -0.013948549022783323, -0.017248935386951603, 0.012094627637729245, -0.012916329790937632, 0.007891048397958124, -0.030776447458492697,

## Vector Stores
- ドキュメントをベクトル化して保存

In [10]:
from langchain.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

# 全ドキュメントをベクトル化すると時間とクレジットを大量に消費しそうんで、リポジトリトップのREADME.mdだけを対象にする
#  -> GPT-3.5 Turbo の 50分の1 ~ 100分の1 の価格なので全部でも大丈夫そう
# 　　-> 時間も1392ドキュメントで十数秒だった
to_vec_docs = docs
# to_vec_docs = []
# for doc in docs:
#     file_path = doc.metadata['file_path']
#     if file_path != 'README.md':
#         continue
#     to_vec_docs.append(doc)
print('docs length:', len(docs), '->', len(to_vec_docs))

# Embeddingモデル
emb_model = OpenAIEmbeddings()

# ベクトル化とVector Storeへの保存
# 実行毎に再構築
db = Chroma.from_documents(
    documents=to_vec_docs,
    embedding=emb_model,
    persist_directory='./my_vec_store',  # 永続化データ保存先(指定しなければ一時的にメモリに保存される)
)

docs length: 1392 -> 1392


## Retrievers
- Vector Storeからユーザー入力の関連度が高いドキュメントを取得

In [11]:
retriever = db.as_retriever()

query = 'LangChainの特徴と使うメリットは？'
context_docs = retriever.get_relevant_documents(query)
print(len(context_docs))

for doc in context_docs:
    print('-'*50)
    print(doc.metadata)
    print(doc.page_content)

2
--------------------------------------------------
{'source': 'docs/docs/modules/model_io/prompts/index.mdx', 'file_path': 'docs/docs/modules/model_io/prompts/index.mdx', 'file_name': 'index.mdx', 'file_type': '.mdx'}
LangChain has a few different types of example selectors you can use off the shelf. You can explore those types [here](./example_selector_types)
--------------------------------------------------
{'source': 'docs/docs/guides/evaluation/index.mdx', 'file_path': 'docs/docs/guides/evaluation/index.mdx', 'file_name': 'index.mdx', 'file_type': '.mdx'}
LangChain offers various types of evaluators to help you measure performance and integrity on diverse data, and we hope to encourage the community to create and share other useful evaluators so everyone can improve. These docs will introduce the evaluator types, how to use them, and provide some examples of their use in real-world scenarios.

Each evaluator type in LangChain comes with ready-to-use implementations and an extens

## RetrievalQA (Chain)
- RAGの一連の流れを実現するChain
- Vector Store(Retriever)は別途作成し与えてあげる必要がある

In [12]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0,
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type='stuff',
    retriever=retriever,
)

result = qa_chain.run(query)
print(result)

/usr/local/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


LangChainの特徴は、さまざまな種類の評価者を提供しており、パフォーマンスやデータの信頼性を測定するのに役立ちます。また、コミュニティが有用な評価者を作成・共有することを奨励しており、誰もが改善できるようにしています。LangChainの評価者タイプには、使用方法や実際のシナリオでの使用例を紹介するドキュメントがあります。また、LangChainの各評価者タイプには、すぐに使用できる実装と、ユニークな要件に合わせてカスタマイズできる拡張可能なAPIが付属しています。
